In [0]:
import random
import numpy as np
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_sets/movie-review-sentiment-analysis-kernels-only/train.tsv', sep="\t")


In [4]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [0]:
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_sets/movie-review-sentiment-analysis-kernels-only/test.tsv', sep="\t")


In [6]:
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [7]:
print(train.shape)
print(test.shape)

(156060, 4)
(66292, 3)


In [8]:
print(train['Phrase'].str.len().mean())
print(train['Phrase'].str.len().max())


40.217224144559786
283


In [9]:
train['Sentiment'].value_counts()


2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
train['Phrase'] = train['Phrase'].apply(lambda x: x.lower())
test['Phrase'] = test['Phrase'].apply(lambda x: x.lower())

In [12]:
train.head(2)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,2,1,a series of escapades demonstrating the adage ...,2


In [13]:
to_categorical(9)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [14]:
train['Sentiment'].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [0]:
X_train = train['Phrase']
X_test = test['Phrase']
y_train = to_categorical(train['Sentiment'].values)

In [0]:
tokenizer= Tokenizer(num_words=10000)


In [0]:
tokenizer.fit_on_texts(list(X_train))

In [0]:
X_train = tokenizer.texts_to_sequences(X_train)

In [0]:
X_train = pad_sequences(X_train, maxlen=283)

In [20]:
print(X_train.shape)
print(y_train.shape)

(156060, 283)
(156060, 5)


In [0]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=283)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test1, y_train, y_test1 = train_test_split(X_train,y_train, test_size=0.25, random_state=101)

In [0]:
from keras.layers import Dense, Embedding,LSTM
from keras.models import Sequential

In [0]:
model = Sequential()


In [0]:
## Embdedding comes in 1st layer

In [0]:
from keras.callbacks import EarlyStopping


In [27]:
model.add(Embedding(10000,100,mask_zero=True))
model.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
#model.add(LSTM(16,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#binary_crossentropy
#categorical_crossentropy
#mean_squared_error

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 1,054,821
Trainable params: 1,054,821
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.callbacks import EarlyStopping
early_stop=[EarlyStopping(monitor='val_acc',mode=max,patience=5)]
model.fit(X_train, y_train, epochs=5, batch_size=12, verbose=1, validation_data=(X_test1, y_test1),callbacks=early_stop)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:511: RuntimeWarning: EarlyStopping mode <built-in function max> is unknown, fallback to auto mode.
  RuntimeWarning)


Train on 117045 samples, validate on 39015 samples
Epoch 1/5
117045/117045 [==============================] - 4641s 40ms/step - loss: 1.0546 - acc: 0.5827 - val_loss: 0.9068 - val_acc: 0.6306
Epoch 2/5
 44952/117045 [==========>...................] - ETA: 45:25 - loss: 0.8551 - acc: 0.6524

In [0]:
X_train[1,282]

In [0]:
y_train.shape